In [ ]:
# Import relevant libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
# Import property data
data = pd.read_excel('Property Data Compiled.xlsx', index_col = 0)

In [ ]:
#  Check rows and column numbers in data
data.shape

In [ ]:
# List of Columns
data.columns.tolist()

## 1. Data Cleaning
Create a subset of original data based on EDA to reduce calculation load. Drop duplicates. Get 5 digit zip & Fix State typo.

In [ ]:
cols = ['PropertyID', 
        # rent fields
        'Avg Effective/SF', 'Avg Concessions %',
        'Studio Effective Rent/SF', 'One Bedroom Effective Rent/SF', 'Two Bedroom Effective Rent/SF',
        'Three Bedroom Effective Rent/SF', 'Four Bedroom Effective Rent/SF',
        # unit fields
        'Studio Avg SF', 'Number Of Studios', 'Studio Vacant Units', 'Studio Vacancy %',
        'One Bedroom Avg SF','Number Of 1 Bedrooms', 'One Bedroom Vacant Units', 'One Bedroom Vacancy %',
        'Two Bedroom Avg SF', 'Number Of 2 Bedrooms', 'Two Bedroom Vacant Units', 'Two Bedroom Vacancy %',
        'Three Bedroom Avg SF', 'Number Of 3 Bedrooms', 'Three Bedroom Vacant Units', 'Three Bedroom Vacancy %',
        'Four Bedroom Avg SF', 'Number Of 4 Bedrooms', 'Four Bedroom Vacant Units', 'Four Bedroom Vacancy %',        
        # location fields
        'State', 'Market Name', 'City', 'Zip', 'County Name',
        'Closest Transit Stop Dist (mi)', 'Latitude', 'Longitude',
        # property fields
        'Star Rating', 'Building Status', 'Land Area (AC)', 'Number Of Stories',
        'Style', 'Number Of Units', 'Vacancy %', 'Avg Unit SF', 'RBA',
        '% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed',
        'Rent Type', 'Affordable Type', 'Construction Material', 
        'Amenities', 'Owner Name', 'Year Built', 'Year Renovated',
        # demographic fields
        '2019 Avg Age(1m)', '2019 Pop Age <19(1m)', '2019 Pop Age 20-64(1m)','2019 Pop Age 65+(1m)']
sub = data.copy()[cols]
sub.drop_duplicates(subset='PropertyID', inplace = True)
sub['Zip5'] = sub['Zip'].str[:5]
sub['State'] = sub['State'].map({'TX':'TX',
                                 'FL':'FL',
                                 'GA':'GA',
                                 'NC':'NC',
                                 'Fl':'FL',
                                 'NC ':'NC'})
sub.shape

In [ ]:
# Export unique zip codes
pd.DataFrame(sub['Zip5'].unique()).to_csv('Zip5.csv',index=False)

In [ ]:
# Check data type and missing values for each of the variables
sub.info()

### 1.1 Drop properties that are proposed, under construction or demolished

In [ ]:
# Number of rows for each building status
sub['Building Status'].value_counts()

In [ ]:
# Remove rows belonging to proposed, under construction and demolished categories
row_initial = sub.shape[0]
sub = sub[sub['Building Status'].isin(['Existing','Under Renovation'])].copy()
print('Records dropped:', row_initial - sub.shape[0])
print('Current size:', sub.shape)

### 1.2 Drop properties that are missing Avg Effective/SF (which is our output variable i.e. final rent price/sqft)
Almost all records missing Avg Effective/SF are missing Effective Rent/SF for specifi unit types too, so cannot be calculated and filled and have to be dropped.

In [ ]:
# Calculate how much data is missing for avg effective/sf
print('Percentage of data missing Avg Effective/SF:',1-sub['Avg Effective/SF'].count()/sub.shape[0])

In [ ]:
# When overal rent/SF is missing, detailed rents by unit types are usually missing too
# The first can't be calculated and filled from the latter
sub[sub['Avg Effective/SF'].isnull()][['Studio Effective Rent/SF', 'One Bedroom Effective Rent/SF', 
                                       'Two Bedroom Effective Rent/SF','Three Bedroom Effective Rent/SF', 
                                       'Four Bedroom Effective Rent/SF']].count()

In [ ]:
# Drop the rows where avg effective/SF is missing
row_initial = sub.shape[0]
sub = sub[sub['Avg Effective/SF'].notnull()].copy()
print('Records dropped:', row_initial - sub.shape[0])
print('Current size:', sub.shape)

### 1.3 Validate property unit mix
### 1.3.1 Fix wrong data in % 4-Bed
Most records have % room types that don't add up to 100. Reason is that Column '% 4-Bed' is not consistent with other % room type columns. Multiply all '% 4-Bed' by 100 to make it consistent.

In [ ]:
# Multiply % 4-Bed by 100 to make data consistent
# Create a % total column to see if all % columns add up to 100
sub['% 4-Bed'] = sub['% 4-Bed']*100
sub['% tot'] = sub[['% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed']].sum(axis=1)
sub['% tot'] = sub['% tot'].apply(round)
sub['% tot'].value_counts()

Some % tot still add up to suggesting missing data. Let's dig deeper into different units and understand this.

### 1.3.2 Fill missing value in % Studios/1-Bed/2-Bed/3-Bed/4-Bed
Some records have Number Of Studios value but no % Studios value. Fill missing value with Number Of Studios divided by Number Of Units. Do the same for 1/2/3/4 bedrooms.

In [ ]:
# Fill missing values in % of unit types with values calculated from other fields
unit_type = {'Studios':'Studios', '1 Bedrooms':'1-Bed', 
             '2 Bedrooms':'2-Bed', '3 Bedrooms':'3-Bed', '4 Bedrooms':'4-Bed'}

for utype in unit_type.keys():
    sub['% {}'.format(unit_type[utype])] =\
    sub[['% {}'.format(unit_type[utype]), 'Number Of {}'.format(utype), 'Number Of Units']].\
    apply(lambda x: x[0] if np.isnan(x[0])==False else(\
    x[0] if np.isnan(x[1]) else x[1]/x[2]*100), axis=1)

### 1.3.3 Fill missing value in Number Of Studios/1-Bed/2-Bed/3-Bed/4-Bed
Some records have % Studios values but no Number Of Studios value. Fill missing value with number of total unit X % Studios. Do the same for 1/2/3/4 bedrooms.

In [ ]:
# Fill missing values in numbers of unit types with values calculated from other fields
for utype in unit_type.keys():
    sub['Number Of {}'.format(utype)] = sub[['Number Of {}'.format(utype), '% {}'.format(unit_type[utype]), 
                                             'Number Of Units']].\
                                        apply(lambda x: x[1] * x[2]/100 if np.isnan(x[0]) else x[0], axis=1)

### 1.3.4 Check if % unit types add up to 100 , fix wrong numbers
Let's come back to % tot to validate number of different units add up.

In [ ]:
# Recheck % total to see if there are still non-100s
sub['% tot'] = sub[['% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed']].sum(axis=1)
sub['% tot'] = sub['% tot'].apply(round)
sub['% tot'].value_counts()

In [ ]:
# Find rows that don't add up to 100 
wrong = sub['% tot'].value_counts().index.tolist()
wrong.remove(100)
sub[sub['% tot'].isin(wrong)][['Studio Effective Rent/SF', 'One Bedroom Effective Rent/SF', 
                               'Two Bedroom Effective Rent/SF','Three Bedroom Effective Rent/SF', 
                               'Four Bedroom Effective Rent/SF',
                               '% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed',
                               'Number Of Studios', 'Number Of 1 Bedrooms', 'Number Of 2 Bedrooms', 
                               'Number Of 3 Bedrooms', 'Number Of 4 Bedrooms', 'Number Of Units']].head(11)

Row 11401 can be easily fixed by removing number of 4 bedrooms and % 4-Bed.

In [ ]:
# Fix row 11401
sub.loc[11401, 'Number Of 4 Bedrooms']=np.nan
sub.loc[11401, '% 4-Bed']=np.nan

Find the rows where number of units add up and fix their % accordingly. Drop the rows where number of units don't add up.

In [ ]:
# Find rows where absolute numbers do and do not add up
test = sub[sub['% tot'].isin(wrong)]
ind = pd.DataFrame(test[['Number Of Studios', 'Number Of 1 Bedrooms', 'Number Of 2 Bedrooms', 
                         'Number Of 3 Bedrooms', 'Number Of 4 Bedrooms']].\
                   sum(axis=1)==test['Number Of Units']).reset_index()
ind.columns=['Row','match']
match = ind[ind['match']==True]['Row'].tolist()
unmatch = ind[ind['match']==False]['Row'].tolist()
print('Matched:', match)
print('Unmatched:', unmatch)

In [ ]:
# Remove unmatched rows
# For matched ones, recalculate the % based on absolute number of units
unit_type = {'Studios':'Studios', '1 Bedrooms':'1-Bed', 
             '2 Bedrooms':'2-Bed', '3 Bedrooms':'3-Bed', '4 Bedrooms':'4-Bed'}
for utype in unit_type.keys():
    sub.loc[match,'% {}'.format(unit_type[utype])] =\
    sub.loc[match, ['% {}'.format(unit_type[utype]), 'Number Of {}'.format(utype), 
                      'Number Of Units']].apply(lambda x: x[1]/x[2]*100, axis=1)
row_initial = sub.shape[0]
sub = sub.drop(unmatch)
print('Records dropped:', row_initial - sub.shape[0])
print('Current size:', sub.shape)

In [ ]:
# Confirm % total all equal to 100 now
sub['% tot'] = sub[['% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed']].sum(axis=1)
sub['% tot'] = sub['% tot'].apply(round)
sub['% tot'].value_counts()

### 1.3.5 Check if % and number of units match
Manually calculate number of different types of units (i.e. # Studios,1/2/3/4 bedrooms) and see if they match with the original data

In [ ]:
# Manually calculate number of unit by types from %
test = sub[['% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed',
                      'Number Of Studios', 'Number Of 1 Bedrooms', 'Number Of 2 Bedrooms', 
                      'Number Of 3 Bedrooms', 'Number Of 4 Bedrooms', 'Number Of Units']].copy()
test = test.fillna(0)
unit_type = {'Studios':'Studios', '1 Bedrooms':'1-Bed', 
             '2 Bedrooms':'2-Bed', '3 Bedrooms':'3-Bed', '4 Bedrooms':'4-Bed'}
for utype in unit_type.keys():
    test['# {}'.format(utype)] = test[['% {}'.format(unit_type[utype]),
                                       'Number Of Units']].apply(lambda x:
                                 round(x[0]*x[1]/100), axis=1)
test.head()

In [ ]:
# Find cases where manually calculated numbers don't match original input
unmatch = \
test[(test['Number Of Studios']!=test['# Studios'])|\
     (test['Number Of 1 Bedrooms']!=test['# 1 Bedrooms'])|\
     (test['Number Of 2 Bedrooms']!=test['# 2 Bedrooms'])|\
     (test['Number Of 3 Bedrooms']!=test['# 3 Bedrooms'])|\
     (test['Number Of 4 Bedrooms']!=test['# 4 Bedrooms'])]
unmatch.head()

In [ ]:
# Percentage mismatch
print("Number of rows where % and number of units don't match:", unmatch.shape[0])
print('Percentage of unmatch:', unmatch.shape[0]/sub.shape[0])

1.2% rows unmatched are acceptable. Use percentage of units for regression model.

### 1.4 Remove rows that have missing values in Amenities & Construction Material
Amenities and construction material variables are critical and filling missing values is difficult for these categories

In [ ]:
# Check number of rows with Amenities and Construction Material information
sub[['Amenities','Construction Material']].count()

In [ ]:
# Delete rows that are missing Amenities and Construction Material information
row_initial = sub.shape[0]
sub = sub[(sub['Amenities'].notnull()) &\
           (sub['Construction Material'].notnull())].copy()
print('Records dropped:', row_initial - sub.shape[0])
print('Current size:', sub.shape)

### 1.5 Check if variables related to each unit type add up
- Fill Avg SF, Vacant Unites & Vacancy % missing values
- Check if # of vacant unit matches vacancy %
- Check if there are outliers/wrong entries

### 1.5.1 Define functions to fill Vacancy% and Avg SF, and calculate Vacant Units

In [ ]:
levels = ['Zip5', 'City', 'County Name', 'Market Name']

# Function to fill vacancy % by unit type missing values with regional mean
def fill_vacancy_mean(uname1, uname2, level):
    sub.loc[sub['% {}'.format(uname1)].notnull(), '{} Vacancy %'.format(uname2)] = \
    sub[sub['% {}'.format(uname1)].notnull()][['{} Vacancy %'.format(uname2), level]].groupby(level).\
    transform(lambda x: x.fillna(x.mean()))

# Function to calculate vacant unit by type from vacancy % to fill missing values
def calculate_vacant_unit(uname1, uname2, uname3):
    sub.loc[sub['% {}'.format(uname1)].notnull(),'{} Vacant Units'.format(uname2)] =\
    sub.loc[sub['% {}'.format(uname1)].notnull(),'{} Vacant Units'.format(uname2)].\
    fillna(sub[sub['% {}'.format(uname1)].notnull()]\
           [['Number Of {}'.format(uname3),'{} Vacancy %'.format(uname2)]].\
           apply(lambda x: round(x[0]*x[1]/100), axis=1))

# Function to fill unit size by unit type missing values with regional mean
def fill_sf_mean(uname1, uname2, level):
    sub.loc[sub['% {}'.format(uname1)].notnull(), '{} Avg SF'.format(uname2)] = \
    sub[sub['% {}'.format(uname1)].notnull()][['{} Avg SF'.format(uname2), level]].groupby(level).\
    transform(lambda x: x.fillna(x.mean()))

### 1.5.2 Studio related variables:

In [ ]:
# Check descriptive statistics of Studio related variables
sub[sub['% Studios'].notnull()][['Studio Avg SF', 'Number Of Studios', 'Studio Vacant Units', 
                                 'Studio Vacancy %', 'Studio Effective Rent/SF', '% Studios']].describe()

Missing values for Avg SF, Vacant Units, Vacancy % and Effective Rent/SF. Unusual small Studio Avg SF - 6. Outliers in Studio Avg SF & Studio Effective Rent/SF.

In [ ]:
# Check for outlier cases where studio avg SF is less than 200
sub[(sub['% Studios'].notnull()) & (sub['Studio Avg SF']<200)]\
[['PropertyID','Studio Effective Rent/SF', 'One Bedroom Effective Rent/SF', 'Two Bedroom Effective Rent/SF',
  'Three Bedroom Effective Rent/SF', 'Four Bedroom Effective Rent/SF', 'Avg Effective/SF', 'Avg Unit SF', 
  '% Studios', '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed',
  'Studio Avg SF', 'Number Of Studios', 'Studio Vacant Units', 'Studio Vacancy %']]

PropertyID 8943094 has wrong Studio Avg SF (6 sf is two small for a studio). Based on other columns, a reasonable fix would be 600.

In [ ]:
# Update Studio Avg SF and Studio effective rent/SF with new values for PropertyID 8943094
sub.loc[sub.PropertyID==8943094, 'Studio Avg SF'] = sub.loc[sub.PropertyID==8943094, 'Studio Avg SF'] * 100
sub.loc[sub.PropertyID==8943094, 'Studio Effective Rent/SF'] = \
sub.loc[sub.PropertyID==8943094, 'Studio Effective Rent/SF'] / 100

In [ ]:
# Fill missing values with predefined functions
for level in levels:
    fill_vacancy_mean('Studios', 'Studio', level)
    fill_sf_mean('Studios', 'Studio', level)
calculate_vacant_unit('Studios', 'Studio', 'Studios')

sub[sub['% Studios'].notnull()][['Studio Avg SF', 'Number Of Studios', 'Studio Vacant Units', 
                                 'Studio Vacancy %', 'Studio Effective Rent/SF', '% Studios']].describe()

In [ ]:
# Check if Studio Vacant units match with Studio Vacancy %
test = sub[sub['% Studios'].notnull()].copy()
test['# Vacant Studio'] = test[['Number Of Studios', 'Studio Vacancy %']].apply(lambda x:
                          round(x[0]*x[1]/100), axis=1)
print(test[test['Studio Vacant Units']!=test['# Vacant Studio']].shape)
#test[test['Studio Vacant Units']!=test['# Vacant Studio']][['Studio Vacant Units', '# Vacant Studio']]

Studio Vacant Units and Studio Vacancy % matches.

### 1.5.2 One Bedroom related variables:

In [ ]:
# Check descriptive statistics of 1-Bedroom related variables
sub[sub['% 1-Bed'].notnull()][['One Bedroom Avg SF','Number Of 1 Bedrooms', 'One Bedroom Vacant Units', 
        'One Bedroom Vacancy %', 'One Bedroom Effective Rent/SF', '% 1-Bed']].describe()

Missing values for Avg SF, Vacant Units, Vacancy % and Effective Rent/SF. Outliers in Avg SF and Effective Rent/SF.

In [ ]:
# Fill missing values with predefined functions
for level in levels:
    fill_vacancy_mean('1-Bed', 'One Bedroom', level)
    fill_sf_mean('1-Bed', 'One Bedroom', level)
calculate_vacant_unit('1-Bed', 'One Bedroom', '1 Bedrooms')

sub[sub['% 1-Bed'].notnull()][['One Bedroom Avg SF','Number Of 1 Bedrooms', 'One Bedroom Vacant Units', 
        'One Bedroom Vacancy %', 'One Bedroom Effective Rent/SF', '% 1-Bed']].describe()

In [ ]:
# Check if 1-Bedroom Vacant units match with 1-Bedroom Vacancy %
test = sub[sub['% 1-Bed'].notnull()].copy()
test['# Vacant 1-Bed'] = test[['Number Of 1 Bedrooms', 'One Bedroom Vacancy %']].apply(lambda x:
                          round(x[0]*x[1]/100), axis=1)
#print(test[test['One Bedroom Vacant Units']!=test['# Vacant 1-Bed']].shape)
test[test['One Bedroom Vacant Units']!=test['# Vacant 1-Bed']]\
[['One Bedroom Vacant Units', '# Vacant 1-Bed']]

One Bedroom Vacant Units and One Bedroom Vacancy % have only one minor mismatch due to rounding and can be ignored

### 1.5.3 Two Bedroom related variables:

In [ ]:
# Check descriptive statistics of 2-Bedroom related variables
sub[sub['% 2-Bed'].notnull()][['Two Bedroom Avg SF','Number Of 2 Bedrooms', 'Two Bedroom Vacant Units', 
        'Two Bedroom Vacancy %', 'Two Bedroom Effective Rent/SF', '% 2-Bed']].describe()

Missing values for Avg SF, Vacant Units, Vacancy % and Effective Rent/SF. Outliers in Avg SF and Effective Rent/SF.

In [ ]:
# Fill missing values with predefined functions
for level in levels:
    fill_vacancy_mean('2-Bed', 'Two Bedroom', level)
    fill_sf_mean('2-Bed', 'Two Bedroom', level)
calculate_vacant_unit('2-Bed', 'Two Bedroom', '2 Bedrooms')

sub[sub['% 2-Bed'].notnull()][['Two Bedroom Avg SF','Number Of 2 Bedrooms', 'Two Bedroom Vacant Units', 
        'Two Bedroom Vacancy %', 'Two Bedroom Effective Rent/SF', '% 2-Bed']].describe()

In [ ]:
# Check if 2-Bedroom Vacant units match with 2-Bedroom Vacancy %
test = sub[sub['% 2-Bed'].notnull()].copy()
test['# Vacant 2-Bed'] = test[['Number Of 2 Bedrooms', 'Two Bedroom Vacancy %']].apply(lambda x:
                          round(x[0]*x[1]/100), axis=1)
print(test[test['Two Bedroom Vacant Units']!=test['# Vacant 2-Bed']].shape)
#print(test[test['Two Bedroom Vacant Units']!=test['# Vacant 2-Bed']]\
#[['Two Bedroom Vacant Units', '# Vacant 2-Bed']].count())

Two Bedroom Vacant Units and Two Bedroom Vacancy % matches.

### 1.5.4 Three Bedroom

In [ ]:
# Check descriptive statistics of 3-Bedroom related variables
sub[sub['% 3-Bed'].notnull()][['Three Bedroom Avg SF','Number Of 3 Bedrooms', 'Three Bedroom Vacant Units', 
        'Three Bedroom Vacancy %', 'Three Bedroom Effective Rent/SF', '% 3-Bed']].describe()

Missing values for Avg SF, Vacant Units, Vacancy % and Effective Rent/SF. Outliers in Avg SF and Effective Rent/SF.

In [ ]:
# Fill missing values with predefined functions
for level in levels:
    fill_vacancy_mean('3-Bed', 'Three Bedroom', level)
    fill_sf_mean('3-Bed', 'Three Bedroom', level)
calculate_vacant_unit('3-Bed', 'Three Bedroom', '3 Bedrooms')

sub[sub['% 3-Bed'].notnull()][['Three Bedroom Avg SF','Number Of 3 Bedrooms', 'Three Bedroom Vacant Units', 
        'Three Bedroom Vacancy %', 'Three Bedroom Effective Rent/SF', '% 3-Bed']].describe()

In [ ]:
# Check if 3-Bedroom Vacant units match with 3-Bedroom Vacancy %
test = sub[sub['% 3-Bed'].notnull()].copy()
test['# Vacant 3-Bed'] = test[['Number Of 3 Bedrooms', 'Three Bedroom Vacancy %']].apply(lambda x:
                          round(x[0]*x[1]/100), axis=1)
print(test[test['Three Bedroom Vacant Units']!=test['# Vacant 3-Bed']].shape)
#print(test[test['Three Bedroom Vacant Units']!=test['# Vacant 3-Bed']]\
#[['Three Bedroom Vacant Units', '# Vacant 3-Bed']].count())

Three Bedroom Vacant Units and Three Bedroom Vacancy % matches.

### 1.5.5 Four Bedroom

In [ ]:
# Check descriptive statistics of 4-Bedroom related variables
sub[sub['% 4-Bed'].notnull()][['Four Bedroom Avg SF','Number Of 4 Bedrooms', 'Four Bedroom Vacant Units', 
        'Four Bedroom Vacancy %', 'Four Bedroom Effective Rent/SF', '% 4-Bed']].describe()

Missing values for Avg SF, Vacant Units, Vacancy % and Effective Rent/SF.  Outliers in Avg SF and Effective Rent/SF.

In [ ]:
# Fill missing values with predefined functions
for level in levels:
    fill_vacancy_mean('4-Bed', 'Four Bedroom', level)
    fill_sf_mean('4-Bed', 'Four Bedroom', level)
calculate_vacant_unit('4-Bed', 'Four Bedroom', '4 Bedrooms')

sub[sub['% 4-Bed'].notnull()][['Four Bedroom Avg SF','Number Of 4 Bedrooms', 'Four Bedroom Vacant Units', 
        'Four Bedroom Vacancy %', 'Four Bedroom Effective Rent/SF', '% 4-Bed']].describe()

In [ ]:
# Check if 4-Bedroom Vacant units match with 4-Bedroom Vacancy %
test = sub[sub['% 4-Bed'].notnull()].copy()
test['# Vacant 4-Bed'] = test[['Number Of 4 Bedrooms', 'Four Bedroom Vacancy %']].apply(lambda x:
                          round(x[0]*x[1]/100), axis=1)
print(test[test['Four Bedroom Vacant Units']!=test['# Vacant 4-Bed']].shape)

Four Bedroom Vacant Units and Four Bedroom Vacancy % matches.

### 1.6 Fill Vacancy % missing values with calculated values
Calculate total vacant units, then divide it by total number of units

In [ ]:
# Check Vacancy % missing values (if < 12908)
sub['Vacancy %'].count()

In [ ]:
# Fill missing values with values calculated from related fields 
sub['#_Vacant_Units'] = sub[['Studio Vacant Units',
                             'One Bedroom Vacant Units',
                             'Two Bedroom Vacant Units',
                             'Three Bedroom Vacant Units',
                             'Four Bedroom Vacant Units']].fillna(0).sum(axis=1)
sub['Vacancy_%'] = sub[['#_Vacant_Units','Number Of Units']].apply(lambda x: round(x[0]/x[1]*100,2), axis=1)
sub['Vacancy %'].fillna(sub['Vacancy_%'], inplace=True)
sub[['Vacancy %','Vacancy_%']].count()

In [ ]:
# Check % mismatch in original vacancy% and calculated vacancy% field
print("Percentage of rows where calculated vacancy % doesn't match original vacancy %:", 
      sub[abs(sub['Vacancy %']-sub['Vacancy_%'])>0.05].shape[0]/sub.shape[0])

### 1.7 Fill Avg Unit SF missing values with calculated values
Calculated as weighted average of the average unit size of differnt unit types

In [ ]:
# Check Avg Unit SF missing values (if < 12908)
sub['Avg Unit SF'].count()

In [ ]:
# Calculate weighted average Unit SF based on related fields
sub['Avg_Unit_SF'] = sub[['% Studios','% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed',
                          'Studio Avg SF', 'One Bedroom Avg SF', 'Two Bedroom Avg SF',
                          'Three Bedroom Avg SF', 'Four Bedroom Avg SF']].fillna(0).\
                     apply(lambda x: round((x[0]*x[5]+x[1]*x[6]+x[2]*x[7]+x[3]*x[8]+x[4]*x[9])/100), axis=1)

In [ ]:
# Fill missing values with calculated  values 
sub['Avg Unit SF'].fillna(sub['Avg_Unit_SF'], inplace=True)
sub[['Avg Unit SF', 'Avg_Unit_SF']].count()

In [ ]:
# Check % mismatch in original Avg Unit SF and calculated Avg Unit SF fields
print("Percentage of rows where calculated Avg Unit SF doesn't match original Avg Unit SF:", 
      sub[abs(sub['Avg Unit SF']-sub['Avg_Unit_SF'])>10].shape[0]/sub.shape[0])

### 1.8 Define functions to fill missing value with mean or median
The functions can be used at different region level (zip code, city, county, market)

In [ ]:
def fill_mean(var, level):
    sub[var] = sub[[level, var]].groupby(level).transform(lambda x: x.fillna(x.mean()))

def fill_median(var, level):
    sub[var] = sub[[level, var]].groupby(level).transform(lambda x: x.fillna(x.median()))

### 1.9 Fill 'Closest Transit Stop Dist (mi)' missing values
- Fill with regional mean
- Manually calculate the remaining missing values

In [ ]:
# Check descriptive statistics of Closest Transit Stop Dist (mi)
sub['Closest Transit Stop Dist (mi)'].describe() # mean close to median

In [ ]:
# Fill missing values
var = 'Closest Transit Stop Dist (mi)'
for level in levels:
    fill_mean(var, level)
sub['Closest Transit Stop Dist (mi)'].count()

Manually find the closest transit and calculate distance.

In [ ]:
# Find the Market Names where no closest transit stops are assigned
# Go back to the original dataset to find the closest transit stops for those markets
question_markets = sub[sub['Closest Transit Stop Dist (mi)'].isnull()]['Market Name'].unique().tolist()
data[data['Market Name'].isin(question_markets)][['Market Name', 'Closest Transit Stop']].drop_duplicates()

Use Palm Beach International as 'Closest Transit Stop' for Port St Lucie/Fort Pierce and Gainesville Regional for Ocala. Calculate 'Closest Transit Stop Dist (mi)'.

In [ ]:
# Define a function using Latitude and Longitude information to calculate the distances manually

# http://www.lat-long.com/Latitude-Longitude-309962-Florida-Palm_Beach_International_Airport.html
PalmBeach = (26.683399, -80.095320) # lat, lon
# https://www.prokerala.com/travel/airports/united-states-of-america/gainesville-regional-airport.html
Gainesville = (29.686, -82.2768)

def calculate_distance(p1, p2):
    lat1 = p1[0]
    lat0 = p2[0]
    lon1 = p1[1]
    lon0 = p2[1]

    # calculation: https://stackoverflow.com/questions/28994289/calculate-euclidean-distance-with-google-maps-coordinates
    deglen = 69 #mile
    x = lat1 - lat0
    yy = lon1 - lon0
    y = (yy) * math.cos(lat0)
    dist = deglen * math.sqrt(x*x + y*y)
    return dist

In [ ]:
# Apply the distance function to fill missing values
sub.loc[sub['Closest Transit Stop Dist (mi)'].isnull(),'Closest Transit Stop Dist (mi)']\
= sub.loc[sub['Closest Transit Stop Dist (mi)'].isnull(),
          ['Closest Transit Stop Dist (mi)', 'Market Name', 'Latitude', 'Longitude']].\
apply(lambda x: x[0] if np.isnan(x[0])==False else\
      (calculate_distance((x[2],x[3]), PalmBeach) if x[1]=='Port St Lucie/Fort Pierce'\
       else calculate_distance((x[2],x[3]), Gainesville)), axis=1)
sub['Closest Transit Stop Dist (mi)'].count()

### 1.10 Fix Land Area (AC) wrong entry, fill missing values with regional median

In [ ]:
# Check descriptive statistics for Land Area (AC)
sub['Land Area (AC)'].describe() # extremely large value!!! use median due to presence of outlier

45318 acres of land area for a rental property is extremely unusual. Resaonable guess would be that it should be square feet instead of acres. 

In [ ]:
# Update wrong entry
acre_to_sf = 43560
sub.loc[sub['Land Area (AC)']==45318, 'Land Area (AC)'] = 45318/acre_to_sf

In [ ]:
# Check land Area extreme values
sub['Land Area (AC)'].sort_values(ascending=False).head(10)

In [ ]:
# Check 99 percentile for Land Area
sub['Land Area (AC)'].quantile(0.99)

In [ ]:
sub['Land Area (AC)'].describe()

Too many outliers, very likely to mess up the model, should be excluded.

In [ ]:
# Fill missing values with regional median
var = 'Land Area (AC)'
for level in levels:
    fill_median(var, level)
sub['Land Area (AC)'].count()

### 1.11 Fill Number Of Stories missing values
- Fill with building style median
- Drop the rows where both number of stories and building style are missing

In [ ]:
# Check descriptive statistics for Number of Stories
sub['Number Of Stories'].describe() # mean and median are close

In [ ]:
# Find building style associated to those properties missing values in Number of Stories
ref = sub[sub['Number Of Stories'].isnull()]['Style']
ref.head()

In [ ]:
# Calculate median number of stories for each style
style_story = sub[['Number Of Stories','Style']].groupby('Style').median()

In [ ]:
# Fill missing values with style median
for ind in ref.index:
    style = sub.loc[ind, 'Style']
    if style in style_story.index:
        sub.loc[ind, 'Number Of Stories'] = style_story.loc[style]['Number Of Stories']
sub['Number Of Stories'].count()

Drop the remaining 5 rows with missing Number Of Stories value because filling it with region mean or median doesn't make sense.

In [ ]:
# Drop rows with no building style information
row_initial = sub.shape[0]
sub = sub[sub['Number Of Stories'].notnull()].copy()
print('Records dropped:', row_initial - sub.shape[0])
print('Current size:', sub.shape)

### 1.12 Fill Year Built missing values with regional median

In [ ]:
# Check descriptive statistics Year Built
sub['Year Built'].describe()

In [ ]:
# Fill missing values with median
var = 'Year Built'
for level in levels:
    fill_median(var, level)
sub['Year Built'].count()

### 1.13 Fill Year Renovated missing values with Year Built

In [ ]:
# Check Year Renovated non null values
sub['Year Renovated'].count()

In [ ]:
# Replace Year Renovated missing values with Year Built
sub['Year Renovated'] = sub['Year Renovated'].fillna(sub['Year Built'])
sub['Year Renovated'].count()

### 1.14 Fill Average Age missing values with regional mean

In [ ]:
# Check description of 2019 Averge Age (1m) - 0 are missing values, fill with mean
sub['2019 Avg Age(1m)'].describe()

In [ ]:
# Find number of rows with missing Average age value
sub[sub['2019 Avg Age(1m)']==0].shape

In [ ]:
# Fill missing values with regional mean
sub['2019 Avg Age(1m)'].replace(0, np.nan, inplace=True)

var = '2019 Avg Age(1m)'
for level in levels:
    fill_mean(var, level)
    
sub['2019 Avg Age(1m)'].describe()

### 1.15 Calculate regional population
- Calculate total population within 1 mile
- 59 missing values (0), fill with median (right skewed distribution)

In [ ]:
# Check non null values for population variables
sub[['2019 Pop Age <19(1m)', '2019 Pop Age 20-64(1m)','2019 Pop Age 65+(1m)']].count()

In [ ]:
# Calculate total population and check descriptive statistics
sub['2019 Pop Tot'] = sub[['2019 Pop Age <19(1m)', '2019 Pop Age 20-64(1m)','2019 Pop Age 65+(1m)']].sum(axis=1)
sub['2019 Pop Tot'].describe() 

In [ ]:
# Checking number of rows where total population is 0 (missing values)
sub[sub['2019 Pop Tot']==0].shape

In [ ]:
# Fill missing values with regional median
sub['2019 Pop Tot'].replace(0, np.nan, inplace=True)

var = '2019 Pop Tot'
for level in levels:
    fill_median(var, level)
    
sub['2019 Pop Tot'].describe()

Some area has very few people (9). Since it's 1 mile radius, can be true.

# 2. External Data

### 2.1 Import Income, marriage % and male/female variables

In [ ]:
# Import demographic data
demo = pd.read_csv('income_marriage.csv')

In [ ]:
demo.head()

In [ ]:
demo.shape

In [ ]:
# Convert Zip to int type in both demo and sub
demo['Zip5'] = demo['Zip5'].astype(int)
sub['Zip5'] = sub['Zip5'].astype(int)

In [ ]:
sub.shape

In [ ]:
# Merge demographic data into property dataset
sub = sub.merge(demo, how='left')
sub.shape

In [ ]:
# Check descriptive statistics and look for missing values
sub[['MedanHHIncome(000)','married %','male/female']].describe()

12 missing values in MedanHHIncome(000), 8 missing values in married % and male/female each.

In [ ]:
# Fill missing values with county mean/median
fill_median('MedanHHIncome(000)', 'County Name')
fill_mean('married %', 'County Name')
fill_mean('male/female', 'County Name')

In [ ]:
# Check if no missing values
sub[['MedanHHIncome(000)','married %','male/female']].count()

### 2.2 Import Per Capita Deposit/Saving variable

In [ ]:
# Import Per Capita Deposit data
deposit = pd.read_csv('per_capita_deposit.csv')

In [ ]:
deposit.head()

In [ ]:
# Check descriptive statistics
print(deposit.shape)
deposit.describe()

In [ ]:
# Fix county name that are named differently
deposit.rename(columns={'County':'County Name'}, inplace=True)
deposit['County Name'] = deposit['County Name'].str.replace('Miami-Dade', 'Miami/Dade')
deposit['County Name'] = deposit['County Name'].str.replace('St. Lucie', 'St Lucie')
deposit['County Name'] = deposit['County Name'].str.replace('McLennan', 'Mclennan')

In [ ]:
# Merge deposit information into property dataset
sub = sub.merge(deposit[['State', 'County Name', 'Deposit (000s) Per Capita']], 
                on=['State', 'County Name',], how='left')
sub.shape

In [ ]:
# Check descriptive statistics after merging
sub['Deposit (000s) Per Capita'].describe()

The geographic area that the property dataset covers in general has higher per capita saving than the four state average.

# 3. Feature Engineering

### 3.1 Create Floor Area Ratio variable

In [ ]:
# Check descriptive statistics for RBA - Rentable Building Area
sub['RBA'].describe()

In [ ]:
# Calculate Floor Area Ratio
acre_to_sf = 43560
sub['Floor Area Ratio'] = sub['RBA']/(sub['Land Area (AC)']*acre_to_sf)
sub['Floor Area Ratio'].describe()

### 3.2 Create Supply variables
- Calculate number of units under same zip code
- Calculate number of vacant units under same zip code

In [ ]:
sub['Supply_all'] = sub[['Number Of Units', 'Zip5']].groupby('Zip5').\
                    transform(lambda x: x.sum())
sub['Supply_vacant'] = sub[['#_Vacant_Units', 'Zip5']].groupby('Zip5').\
                       transform(lambda x: x.sum())

Extreme outliers exit, should be excluded.

### 3.3 Create Owner Type, Encode
- Large owner: manages >=50 properties
- Medium owner: manages [10,50) properties
- Small owner: manages <10 properties or unspecified owner
- One hot encode Owner Type

In [ ]:
sub['Owner Name'].nunique()

In [ ]:
# Create a datafame to store property count for each owner
owner = sub['Owner Name'].value_counts().reset_index(name='count')

In [ ]:
print('large owner:', owner[owner['count']>=50].shape[0])
print('medium owner:', owner[(owner['count']>=10) & (owner['count']<50)].shape[0])
print('small owner:', owner[owner['count']<10].shape[0])

In [ ]:
# Create owner lists by owner type
large_owner = owner[owner['count']>=50]['index'].tolist()
medium_owner = owner[(owner['count']>=10) & (owner['count']<50)]['index'].tolist()
small_owner = owner[owner['count']<10]['index'].tolist()

In [ ]:
# Assign owner type to each property
# those who are missing owner info considered as small owner
sub['Owner Type'] = sub['Owner Name'].apply(lambda x: 'large' if x in large_owner else(
                                        'medium' if x in medium_owner else 'small'))

In [ ]:
# Owner Type summary
sub[['Owner Type', 'Avg Effective/SF']].groupby('Owner Type').mean().reset_index().\
merge(pd.DataFrame(sub['Owner Type'].value_counts().reset_index()).\
rename(columns={'index':'Owner Type', 'Owner Type':'Count'}))

In [ ]:
# CreatE dummy variables
for otype in sub['Owner Type'].unique():
    sub['Owner Type_{}'.format(otype)] = sub['Owner Type'].apply(lambda x: 1 if x==otype else 0)

### 3.4 Affordable Housing Encoding
- Combine Rent Stabilized and Rent Controlled into Rent Restricted, label non-affordable properties as Market
- One hot encode Affordable Type

In [ ]:
sub['Rent Type'].count()

In [ ]:
sub['Affordable Type'].value_counts()

In [ ]:
# Combining Rent Stabilized and Controlled as Rent Restricted and fill NULL values as Market
sub['Affordable Type*'] = sub['Affordable Type'].fillna('Market')
sub['Affordable Type*'] = sub['Affordable Type*'].map({'Rent Stabilized':'Rent Restricted',
                                                       'Rent Controlled':'Rent Restricted',
                                                       'Rent Restricted':'Rent Restricted',
                                                       'Market':'Market',
                                                       'Rent Subsidized':'Rent Subsidized',
                                                       'Affordable Units':'Affordable Units'})

In [ ]:
# Affordable Type Summary
sub[['Affordable Type*', 'Avg Effective/SF']].groupby('Affordable Type*').mean().reset_index().\
merge(pd.DataFrame(sub['Affordable Type*'].value_counts().reset_index()).\
rename(columns={'index':'Affordable Type*', 'Affordable Type*':'Count'})).\
sort_values('Avg Effective/SF', ascending=False)

In [ ]:
# Create Dummy Variables
for atype in sub['Affordable Type*'].unique():
    sub['Affordable Type_{}'.format(atype)] = sub['Affordable Type*'].apply(lambda x: 1 if x==atype else 0)

### 3.5 State and City Encoding
- One hot encode State
- Group cities with <= 100 properties in it into Other by state, resulting in 23 city groups
- One hot encode City

In [ ]:
# State summary
sub[['State', 'Avg Effective/SF']].groupby('State').mean().reset_index().\
merge(pd.DataFrame(sub['State'].value_counts().reset_index()).\
rename(columns={'index':'State', 'State':'Count'})).\
sort_values('Avg Effective/SF', ascending=False)

In [ ]:
# Create Dummy Variables for state
for stype in sub['State'].unique():
    sub['State_{}'.format(stype)] = sub['State'].apply(lambda x: 1 if x==stype else 0)

In [ ]:
sub['City'].nunique()

In [ ]:
# Check number of cities with more than 100 properties
city = sub['City'].value_counts().reset_index()
city.columns = ['City', 'Count']
city[city['Count']>100].shape

In [ ]:
# Cities with less than 10 properties
print('{}% cities have less than 10 properties in it'.\
      format(round(city[city['Count']<10].shape[0]/sub['City'].nunique()*100,1)))

In [ ]:
# Combine cities with fewer than 100 properties for each state
large_city = city[city['Count']>100]['City'].unique()
sub['City*'] = sub[['City','State']].apply(lambda x: x[0] if x[0] in large_city\
                                          else '{} Other'.format(x[1]), axis=1)

In [ ]:
sub['City*'].nunique()

In [ ]:
# Check if the smallest cities have enough property count
sub['City*'].value_counts().tail()

In [ ]:
# Create dummy variables for cities
for ctype in sub['City*'].unique():
    sub['City_{}'.format(ctype)] = sub['City*'].apply(lambda x: 1 if x==ctype else 0)

### 3.6 Construction Material
- Combine Steel and Metal into Steel or Metal
- One hot encode Construction Material

In [ ]:
sub['Construction Material'].value_counts()

In [ ]:
# Combine Steel and Metal
sub['Construction Material*'] = sub['Construction Material'].map({'Wood Frame':'Wood Frame',
                                                                    'Masonry':'Masonry',
                                                                    'Reinforced Concrete':'Reinforced Concrete',
                                                                    'Steel':'Steel or Metal',
                                                                    'Metal':'Steel or Metal'})

In [ ]:
# Construction Material summary
sub[['Construction Material*', 'Avg Effective/SF']].groupby('Construction Material*').mean().reset_index().\
merge(pd.DataFrame(sub['Construction Material*'].value_counts().reset_index()).\
rename(columns={'index':'Construction Material*', 'Construction Material*':'Count'})).\
sort_values('Avg Effective/SF', ascending=False)

In [ ]:
# Create dummy variables for construction material
for material in sub['Construction Material*'].unique():
    sub['Construction Material_{}'.format(material)] = \
    sub['Construction Material*'].apply(lambda x: 1 if x==material else 0)

### 3.7 Amenities w/o grouping

In [ ]:
# Parse all amenities
amenities = {}
all_amenities = sub['Amenities'].str.split(', ').tolist()
for row in all_amenities:
    #print(row)
    for item in row:
        #print(item)
        if item not in amenities.keys():
            amenities[item] = 1
        else:
            amenities[item] += 1
len(amenities)

In [ ]:
# Calculate rent with or without a given amenity
amenity_vs_rent = pd.DataFrame()

for amenity in amenities:
    yes = sub[sub['Amenities'].str.contains(amenity)]['Avg Effective/SF'].mean()
    no = sub[-sub['Amenities'].str.contains(amenity)]['Avg Effective/SF'].mean()
    count = sub[sub['Amenities'].str.contains(amenity)]['Avg Effective/SF'].count()
    amenity_vs_rent = amenity_vs_rent.append(pd.DataFrame([amenity,count,yes,no,yes-no]).T,
                                             ignore_index=True)

amenity_vs_rent.columns = ['amenity','count','yes', 'no', 'diff']
amenity_vs_rent = amenity_vs_rent.sort_values('diff')
amenity_vs_rent.tail()

In [ ]:
# Top 15 most popular amenities
amenity_vs_rent.sort_values('count', ascending=False).head(15)

In [ ]:
# Plotting rent difference with vs without amenities
test = amenity_vs_rent[amenity_vs_rent['amenity']!='Study Lounge']
xticklabel = test[['amenity','count']].apply(lambda x: x[0]+' '+str(x[1]), axis=1)

plt.figure(figsize=(20,10))
plt.bar(np.arange(test.shape[0]),
        test['diff'])
plt.xticks(np.arange(test.shape[0]),
           xticklabel,
           rotation=90)
plt.yticks(rotation=90)
plt.axhline(sub['Avg Effective/SF'].std(),
            color='red',
            linestyle='--')
plt.annotate('Average\nEffective Rent/SF\nStd', 
             (0,0.5), fontsize=12, color='red',
             rotation=90)
#plt.title('Average Effective Rent/SF With vs Without Amenity', fontsize=15)
plt.ylabel('Difference in Average Effective Rent/SF', fontsize=15)
plt.show()

#### Group Amenities
- LEED Certified - Silver, LEED Certified - Gold, LEED Certified, LEED Certified - Platinum, Energy Star Labeled
- Sports: Tennis Court, Volleyball Court, Basketball Court
- Business: Business Center, Corporate Suites, Confere Rooms, Multi Use Room
- Laundry: Laundry Facilities, Laundry Service
- Spa: Spa, Sauna
- Pet: Pet Washing Station, Pet Care, Pet Play Area
- Wifi: Community-Wide WiFi, Wi-Fi

#### + Less popular high impact amenities:
- Roof Terrace
- Maid Service
- Bicycle Storage
- Car Charging Station	
- On-Site Retail
- Elevator

#### + Top 15 popular amenities

In [ ]:
# Total Amenities after grouping and adding popular amenities
amen = ['LEED Certified - Silver', 'LEED Certified - Gold', 'LEED Certified', 'LEED Certified - Platinum', 
        'Energy Star Labeled',
        # Sports
        'Tennis Court', 'Volleyball Court', 'Basketball Court',
        # Business
        'Business Center', 'Corporate Suites', 'Confere Rooms', 'Multi Use Room',
        # Laundry
        'Laundry Facilities', 'Laundry Service',
        # Spa
        'Spa', 'Sauna',
        # Pet
        'Pet Washing Station', 'Pet Care', 'Pet Play Area',
        # Wifi
        'Community-Wide WiFi', 'Wi-Fi',
        # Less popular high impact
        'Roof Terrace', 'Maid Service', 'Bicycle Storage',
        'Car Charging Station', 'On-Site Retail', 'Elevator']
top_15_amenities = amenity_vs_rent.sort_values('count', ascending=False).head(15)['amenity'].tolist()

for amenity in top_15_amenities:
    if amenity not in amen:
        amen.append(amenity)

len(amen)

In [ ]:
# One hot encode selected amenites
# Group encoded amenities and discard original ones
for amenity in amen:
    sub['Amenity_{}'.format(amenity)] = sub['Amenities'].apply(lambda x: 1 if amenity in x.split(', ') else 0)

sub['Amenity_LEED/Energy Star'] = sub[['Amenity_LEED Certified - Silver', 
                                       'Amenity_LEED Certified - Gold', 
                                       'Amenity_LEED Certified', 
                                       'Amenity_LEED Certified - Platinum', 
                                       'Amenity_Energy Star Labeled']].max(axis=1)
sub['Amenity_Sports'] = sub[['Amenity_Tennis Court', 
                             'Amenity_Volleyball Court', 
                             'Amenity_Basketball Court']].max(axis=1)
sub['Amenity_Business'] = sub[['Amenity_Business Center', 
                               'Amenity_Corporate Suites', 
                               'Amenity_Confere Rooms', 
                               'Amenity_Multi Use Room']].max(axis=1)
sub['Amenity_Laundry'] = sub[['Amenity_Laundry Facilities', 
                              'Amenity_Laundry Service']].max(axis=1)
sub['Amenity_Spa/Sauna'] = sub[['Amenity_Spa', 
                                'Amenity_Sauna']].max(axis=1)
sub['Amenity_Pet'] = sub[['Amenity_Pet Washing Station', 
                          'Amenity_Pet Care', 
                          'Amenity_Pet Play Area']].max(axis=1)
sub['Amenity_Wifi'] = sub[['Amenity_Community-Wide WiFi', 
                           'Amenity_Wi-Fi']].max(axis=1)
cols = ['Amenity_LEED Certified - Silver', 'Amenity_LEED Certified - Gold', 'Amenity_LEED Certified', 
        'Amenity_LEED Certified - Platinum', 'Amenity_Energy Star Labeled',
        'Amenity_Tennis Court', 'Amenity_Volleyball Court', 'Amenity_Basketball Court',
        'Amenity_Business Center', 'Amenity_Corporate Suites', 
        'Amenity_Confere Rooms', 'Amenity_Multi Use Room',
        'Amenity_Laundry Facilities', 'Amenity_Laundry Service',
        'Amenity_Spa', 'Amenity_Sauna',
        'Amenity_Pet Washing Station', 'Amenity_Pet Care', 'Amenity_Pet Play Area',
        'Amenity_Community-Wide WiFi', 'Amenity_Wi-Fi']
sub.drop(columns=cols, inplace=True)

In [ ]:
# Amenity vs. Avg effective/SF (when amenity is present vs. not present)
cols = ['Amenity_Roof Terrace', 'Amenity_Maid Service', 'Amenity_Bicycle Storage',
       'Amenity_Car Charging Station', 'Amenity_On-Site Retail', 'Amenity_Elevator',
       'Amenity_Fitness Center', 'Amenity_Clubhouse', 'Amenity_Property Manager on Site',
       'Amenity_Grill', 'Amenity_Playground', 'Amenity_Maintenance on site', 'Amenity_Gated',
       'Amenity_Picnic Area', 'Amenity_Package Service', 'Amenity_Controlled Access',
       'Amenity_Sundeck', 'Amenity_LEED/Energy Star', 'Amenity_Sports', 'Amenity_Business',
       'Amenity_Laundry', 'Amenity_Spa/Sauna', 'Amenity_Pet', 'Amenity_Wifi']
amenity_summary = pd.DataFrame(sub[cols].sum()).rename(columns={0:'# Properties'})
amenity_summary['% Properties'] = amenity_summary['# Properties']/sub.shape[0]*100
amenity_summary['% Properties'] = amenity_summary['% Properties'].apply(lambda x: round(x,1))
for amenity in cols:
    amenity_summary.loc[amenity, 'Avg Effective/SF w/ Amenity'] = \
    sub[sub[amenity]==1]['Avg Effective/SF'].mean()
    amenity_summary.loc[amenity, 'Avg Effective/SF w/o Amenity'] = \
    sub[sub[amenity]==0]['Avg Effective/SF'].mean()
amenity_summary['Avg Effective/SF Diff'] = amenity_summary['Avg Effective/SF w/ Amenity']-\
                                           amenity_summary['Avg Effective/SF w/o Amenity']
amenity_summary = amenity_summary.sort_values('Avg Effective/SF Diff', ascending=False)
amenity_summary

# 4. Linear Regression

In [ ]:
# Final list of variables considered for regression model (Base group excluded for analsysi)
cols = ['PropertyID', 'Avg Effective/SF', 
       # Property
       'Avg Concessions %', 'Vacancy %', 'Avg Unit SF', 'Year Built', 'Year Renovated', 'Star Rating', 
       'Number Of Units', 'RBA', 'Floor Area Ratio', 'Land Area (AC)', 'Number Of Stories', 
       '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed', 
       'Construction Material_Masonry', 'Construction Material_Reinforced Concrete',
       'Construction Material_Steel or Metal',
       'Owner Type_large', 'Owner Type_medium',
       'Affordable Type_Rent Restricted', 'Affordable Type_Rent Subsidized', 'Affordable Type_Affordable Units',
       # Location & Demographic
       'Closest Transit Stop Dist (mi)', '2019 Avg Age(1m)', '2019 Pop Tot', 
       'MedanHHIncome(000)', 'married %', 'male/female', 'Deposit (000s) Per Capita',
       'Supply_all', 'Supply_vacant',
       'State_GA', 'State_FL', 'State_NC',
       'City_Atlanta', 'City_Dallas', 'City_Tampa', 'City_Orlando', 'City_Miami',
       'City_Jacksonville', 'City_Tallahassee', 'City_Charlotte', 'City_Durham',
       'City_Greensboro', 'City_Raleigh', 'City_Fort Worth', 'City_San Antonio',
       'City_Austin', 'City_Houston', 'City_Arlington', 'City_El Paso', 'City_Irving', 'City_Plano',
       # Amenities
       'Amenity_Roof Terrace', 'Amenity_Maid Service', 'Amenity_Bicycle Storage',
       'Amenity_Car Charging Station', 'Amenity_On-Site Retail', 'Amenity_Elevator',
       'Amenity_Fitness Center', 'Amenity_Clubhouse', 'Amenity_Property Manager on Site',
       'Amenity_Grill', 'Amenity_Playground', 'Amenity_Maintenance on site', 'Amenity_Gated',
       'Amenity_Picnic Area', 'Amenity_Package Service', 'Amenity_Controlled Access',
       'Amenity_Sundeck', 'Amenity_LEED/Energy Star', 'Amenity_Sports', 'Amenity_Business',
       'Amenity_Laundry', 'Amenity_Spa/Sauna', 'Amenity_Pet', 'Amenity_Wifi']
final_onehot = sub[cols].copy()
final_onehot = final_onehot.fillna(0)
final_onehot.shape

### 4.1 Export for JMP

In [ ]:
final_onehot.to_csv('final_onehot_1125.csv', index=False)

### 4.2 Statsmodels
- The result matches JMP result
- Further variable and outlier exclusions are done in JMP

In [ ]:
test = final_onehot.copy()

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import OLSInfluence as infl

Check VIF to ensure no severe multicolinearity.

In [ ]:
Xc = add_constant(test.iloc[:,2:])
vifs = [vif(Xc.values, i) for i in range(len(Xc.columns))]
pd.Series(data=vifs, index=Xc.columns).sort_values(ascending=False)

In [ ]:
model = sm.OLS(test['Avg Effective/SF'], Xc)

In [ ]:
result = model.fit()

In [ ]:
print(result.summary())

# 5. Random Forest Regressor with Scikit-Learn

### 5.1 Exclude outliers identified from the regression model

In [ ]:
jmp = pd.read_csv('final_onehot_1125_outlier_est_CI.dat')

In [ ]:
jmp.shape

In [ ]:
outlier = jmp[jmp['Outlier']==1]['PropertyID'].tolist()
len(outlier)

In [ ]:
cols = ['PropertyID', 'Avg Effective/SF', 
       # Property
       'Avg Concessions %', 'Vacancy %', 'Avg Unit SF', 'Year Built', 'Year Renovated', 'Star Rating', 
       'Number Of Units', 'RBA', 'Floor Area Ratio', 'Land Area (AC)', 'Number Of Stories', 
       '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed', 
       'Construction Material*', 'Owner Type', 'Affordable Type*',
       # Location & Demographic
       'Closest Transit Stop Dist (mi)', '2019 Avg Age(1m)', '2019 Pop Tot',
       'MedanHHIncome(000)', 'married %', 'male/female', 'Deposit (000s) Per Capita',
       'Supply_all', 'Supply_vacant',
       'State', 'City', 'County Name',
       # Amenities
       'Amenity_Roof Terrace', 'Amenity_Maid Service', 'Amenity_Bicycle Storage',
       'Amenity_Car Charging Station', 'Amenity_On-Site Retail', 'Amenity_Elevator',
       'Amenity_Fitness Center', 'Amenity_Clubhouse', 'Amenity_Property Manager on Site',
       'Amenity_Grill', 'Amenity_Playground', 'Amenity_Maintenance on site', 'Amenity_Gated',
       'Amenity_Picnic Area', 'Amenity_Package Service', 'Amenity_Controlled Access',
       'Amenity_Sundeck', 'Amenity_LEED/Energy Star', 'Amenity_Sports', 'Amenity_Business',
       'Amenity_Laundry', 'Amenity_Spa/Sauna', 'Amenity_Pet', 'Amenity_Wifi']
sub1 = sub[-sub['PropertyID'].isin(outlier)][cols].copy()
sub1 = sub1.fillna(0)
sub1.shape

### 5.2 Regroup City, County and Affordable Type
- Make sure test subset will have enough observations in each category

In [ ]:
city = sub1['City'].value_counts().reset_index()
city.columns = ['City', 'Count']
large_city = city[city['Count']>200]['City'].unique()
sub1['City**'] = sub1[['City','State']].apply(lambda x: x[0] if x[0] in large_city\
                                          else '{} Other'.format(x[1]), axis=1)
print('# City:', sub1['City**'].nunique())
print('Smallest city bin:', sub1['City**'].value_counts().tail(1))

In [ ]:
county = sub1['County Name'].value_counts().reset_index()
county.columns = ['County', 'Count']
large_county = county[county['Count']>200]['County'].unique()
sub1['County**'] = sub1[['County Name','State']].apply(lambda x: x[0] if x[0] in large_county\
                                          else '{} Other'.format(x[1]), axis=1)
print('# County:', sub1['County**'].nunique())
print('Smallest county bin:', sub1['County**'].value_counts().tail(1))

In [ ]:
sub1['Construction Material*'].value_counts()

Although Steel or Metal only have 134 observations, it's hard to reason combining it with any other categories, so will leave it as it is.

In [ ]:
sub1['Owner Type'].value_counts()

In [ ]:
sub1['Affordable Type*'].value_counts()

In [ ]:
sub1['Affordable Type**'] = sub1['Affordable Type*'].map({'Rent Restricted':'Rent Restricted',
                                                          'Market':'Market',
                                                          'Rent Subsidized':'Rent Subsidized/Affordable Units',
                                                          'Affordable Units':'Rent Subsidized/Affordable Units'})

In [ ]:
sub1['Affordable Type**'].value_counts()

In [ ]:
cols = ['PropertyID', 'Avg Effective/SF', 
       # Property
       'Avg Concessions %', 'Vacancy %', 'Avg Unit SF', 'Year Built', 'Year Renovated', 'Star Rating', 
       'Number Of Units', 'RBA', 'Floor Area Ratio', 'Land Area (AC)', 'Number Of Stories', 
       '% 1-Bed', '% 2-Bed', '% 3-Bed', '% 4-Bed', 
       'Construction Material*', 'Owner Type', 'Affordable Type**',
       # Location & Demographic
       'Closest Transit Stop Dist (mi)', '2019 Avg Age(1m)', '2019 Pop Tot',
       'MedanHHIncome(000)', 'married %', 'male/female', 'Deposit (000s) Per Capita',
       'Supply_all', 'Supply_vacant',
       'State', 'City**', 'County**',
       # Amenities
       'Amenity_Roof Terrace', 'Amenity_Maid Service', 'Amenity_Bicycle Storage',
       'Amenity_Car Charging Station', 'Amenity_On-Site Retail', 'Amenity_Elevator',
       'Amenity_Fitness Center', 'Amenity_Clubhouse', 'Amenity_Property Manager on Site',
       'Amenity_Grill', 'Amenity_Playground', 'Amenity_Maintenance on site', 'Amenity_Gated',
       'Amenity_Picnic Area', 'Amenity_Package Service', 'Amenity_Controlled Access',
       'Amenity_Sundeck', 'Amenity_LEED/Energy Star', 'Amenity_Sports', 'Amenity_Business',
       'Amenity_Laundry', 'Amenity_Spa/Sauna', 'Amenity_Pet', 'Amenity_Wifi']
var = sub1[cols].copy()

### 5.3 Separate train and test sets
- 0.7 train and 0.3 test to make sure test set have enough observations in each category

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(var, test_size=0.3, random_state=0)
train = train.copy()
test = test.copy()

In [ ]:
print(train.shape)
print(test.shape)

### 5.4 Target encode six categorical features

In [ ]:
# 'Construction Material*', 'Owner Type', 'Affordable Type**', 'State', 'City**', 'County**'

In [ ]:
train['Construction Material_encoded'] = train[['Construction Material*','Avg Effective/SF']].\
                                         groupby('Construction Material*').transform(lambda x: x.mean())
test['Construction Material_encoded'] = test[['Construction Material*','Avg Effective/SF']].\
                                        groupby('Construction Material*').transform(lambda x: x.mean())
    
train['Owner Type_encoded'] = train[['Owner Type','Avg Effective/SF']].\
                              groupby('Owner Type').transform(lambda x: x.mean())
test['Owner Type_encoded'] = test[['Owner Type','Avg Effective/SF']].\
                              groupby('Owner Type').transform(lambda x: x.mean())

train['Affordable Type_encoded'] = train[['Affordable Type**','Avg Effective/SF']].\
                              groupby('Affordable Type**').transform(lambda x: x.mean())
test['Affordable Type_encoded'] = test[['Affordable Type**','Avg Effective/SF']].\
                              groupby('Affordable Type**').transform(lambda x: x.mean())

train['State_encoded'] = train[['State','Avg Effective/SF']].\
                              groupby('State').transform(lambda x: x.mean())
test['State_encoded'] = test[['State','Avg Effective/SF']].\
                              groupby('State').transform(lambda x: x.mean())

train['City_encoded'] = train[['City**','Avg Effective/SF']].\
                              groupby('City**').transform(lambda x: x.mean())
test['City_encoded'] = test[['City**','Avg Effective/SF']].\
                              groupby('City**').transform(lambda x: x.mean())

train['County_encoded'] = train[['County**','Avg Effective/SF']].\
                              groupby('County**').transform(lambda x: x.mean())
test['County_encoded'] = test[['County**','Avg Effective/SF']].\
                              groupby('County**').transform(lambda x: x.mean())

In [ ]:
cols = ['Construction Material*', 'Owner Type', 'Affordable Type**', 'State', 'City**', 'County**']
train = train.drop(columns=cols)
test = test.drop(columns=cols)

In [ ]:
X_train = train.iloc[:, 2:].values
y_train = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values
y_test = test.iloc[:, 1].values

### 5.5 Model fitting with 54 features

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(X_train, y_train)
y_pred_train = regressor.predict(X_train)
y_pred_test = regressor.predict(X_test)

In [ ]:
from sklearn import metrics

print('------------------------------TRAIN------------------------------')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))
print('R Squared:', metrics.r2_score(y_train, y_pred_train))
print('')
print('------------------------------TEST------------------------------')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))
print('R Squared:', metrics.r2_score(y_test, y_pred_test))

High Overfitting. Requires future work.

# 6. Identify Underpriced Properties

In [ ]:
pred = jmp[['PropertyID', 'Avg Effective/SF', 'Pred Formula Avg Effective/SF',
            'Lower 95% Indiv Avg Effective/SF', 'Upper 95% Indiv Avg Effective/SF', 'Outlier']].copy()
pred.head()

In [ ]:
pred['Overpriced'] = pred[['Avg Effective/SF', 'Upper 95% Indiv Avg Effective/SF']].\
                     apply(lambda x: 1 if x[0]>x[1] else 0, axis=1)
pred['Underpriced'] = pred[['Avg Effective/SF', 'Lower 95% Indiv Avg Effective/SF']].\
                     apply(lambda x: 1 if x[0]<x[1] else 0, axis=1)

In [ ]:
pred[['Overpriced', 'Underpriced']].sum()

In [ ]:
pred[pred['Outlier']==0][['Overpriced', 'Underpriced']].sum()

In [ ]:
underpriced = pred[(pred['Outlier']==0) & (pred['Underpriced']==1)]\
              [['PropertyID', 'Pred Formula Avg Effective/SF']]
underpriced = underpriced.merge(sub[['PropertyID', 'State', 'Latitude', 'Longitude', 'Avg Effective/SF']])
underpriced.head()

In [ ]:
underpriced.to_csv('underpriced.csv',index=False)